
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# LAB - Building Multi-stage AI System

In this lab, you will construct a multi-stage reasoning system using Databricks' features and LangChain.

You will start by building the first chain, which performs a search using a dataset containing product descriptions from Etsy. Following that, you will create the second chain, which creates an image for the proposed product. Finally, you will integrate these chains to form a complete multi-stage AI system.


**Lab Outline:**

In this lab, you will need to complete the following tasks;

* **Task 1:** Create a Vector Store

* **Task 2:** Build the First Chain (Vector Store Search)

* **Task 3:** Build the Second Chain (Product Image)

* **Task 4:**  Integrate Chains into a Multi-chain System

**📝 Your task:** Complete the **`<FILL_IN>`** sections in the code blocks and follow the other steps as instructed.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **15.4.x-cpu-ml-scala2.12**


## Classroom Setup

Before starting the demo, run the provided classroom setup script. This script will define configuration variables necessary for the demo. Execute the following cell:

In [0]:
%pip install -U -qq databricks-sdk databricks-vectorsearch langchain-databricks langchain==0.3.7 langchain-community==0.3.7

dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup-02LAB

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.



The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmful content.


2025-04-30 15:22:52,975 14883 ERROR _handle_rpc_error GRPC Error received
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/client/core.py", line 1721, in config
    resp = self._stub.Config(req, metadata=self.metadata())
  File "/databricks/python/lib/python3.10/site-packages/grpc/_interceptor.py", line 277, in __call__
    response, ignored_call = self._with_call(
  File "/databricks/python/lib/python3.10/site-packages/grpc/_interceptor.py", line 332, in _with_call
    return call.result(), call
  File "/databricks/python/lib/python3.10/site-packages/grpc/_channel.py", line 439, in result
    raise self
  File "/databricks/python/lib/python3.10/site-packages/grpc/_interceptor.py", line 315, in continuation
    response, call = self._thunk(new_method).with_call(
  File "/databricks/python/lib/python3.10/site-packages/grpc/_channel.py", line 1193, in with_call
    return _end_unary_response_blocking(state, call, True, None)
 

**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

Username:          labuser10152510_1746020930@vocareum.com
Catalog Name:      dbacademy
Schema Name:       labuser10152510_1746020930
Working Directory: /Volumes/dbacademy/ops/labuser10152510_1746020930@vocareum_com
Dataset Location:  NestedNamespace (dais='/Volumes/dbacademy_dais/v01')


## Load Dataset

Before you start building the AI chain, you will need to load dataset and save to a Delta table. For this demo, we will use **[Bright Data's Etsy Dataset](/marketplace/consumer/listings/9e69c469-394d-4c7f-8f06-38a2139143a5?o=2659148800588914)** listed in Databricks Marketplace. Only product title and description will be used. The table will be created for you in the next code block.

In [0]:
# create product text table
vs_source_table_fullname = f"{DA.catalog_name}.{DA.schema_name}.product_text"
create_product_text_table(vs_source_table_fullname)

# show table
display(spark.sql(f"SELECT * FROM {vs_source_table_fullname}"))

Validation of table bright_data_etsy_dataset.datasets.etsy complete. No errors found.


id,product
1001239907,"## Title: Boho Wedding Guest Book. Gold Foil Wedding Guestbook. Boho Wedding Gift Idea. Real Foil Guest Book. Terracotta Personalized Guest Book. KP3A ## Description: Welcome to Drift House Press ~ Please check out our full collection here https://www.etsy.com/shop/DriftHousePress G O L D . F O I L . W E D D I N G . G U E S T . B O O K Our luxury wedding guestbooks are created for brides & grooms-to-be who value unique and thoughtful design. These high end guest books are the perfect way to elevate your wedding reception and tie in your color palette. - H O W . I T . W O R K S . PLACE YOUR ORDER Choose the size of guest book, page quantity, cover and foil color. Add your unique details to the personalisation box provided and place your order. . DESIGN + PROOF We will work our magic and create your unique book design. You will receive a proof for inspection within 1-2 business days. You can let us know then if any changes are required. . APPROVAL + PRODUCTION Once you are 100% happy with the finished result we will finalise the guest book for production. Production time (time to make the books) can take between 5-7 business days. . SHIPPING When your book is complete it will be dispatched, you will receive a shipping notification. Shipping generally takes between 2-3 business days. . ENJOY Receive and cherish your book! - A B O U T . T H E . B O O K S . BOOK SIZES Our guestbooks come in 3 size options - 9x6"", 10x8"", 12x12"" . COVERS Every guest book is has a perfect bound hardcover with a soft to the touch, velvet matte finish. . FOIL Luxury raised foiling is available in gold, silver and rose gold finishes. . INNER PAGES You can choose between 50 or 100 pages. White, Lined or Black. - W H Y . C H O O S E . U S We blend clean, timeless design with our signature color palette, luxury gold foiling and the highest quality book bindery to create meaningful keepsakes for your wedding. Thank you for letting us play a small part in your story, we’re so happy you are part of ours. - I M P O R T A N T . I N F O . Colors may vary depending on monitor, we do our absolute best to accurately represent colors on screen. - If you have any questions at all please contact us through the 'contact shop owner' button to find out everything you need to know prior to ordering. Original Designs by Ruth Harvey | © 2013-Present Kismet Weddings Ltd. Learn more about this item"
195137531,## Title: M bridal earrings jewelry bridesmaid gift wedding Clear white teardrop cubic zirconia teardrop cz rhodium post earrings - Free US shipping ## Description: Tarnish resistant White gold plated metal framed AAA high quality clear white cubic zirconia teardrop post earrings with AAA quality teardrop clear white cubic zirconia pendant. Earring posts are surgical steel.. Size: 2.7cm x 1.1cm Matching necklace available. Learn more about this item
1336040522,"## Title: Acrylic Married Ornament Gift Newlywed Gift Mr & Mrs Christmas Ornament Personalized Mr Mrs Wedding Ornament Wedding Gift Keepsake No.343 ## Description: Make your family memories bright with Hello Delicious beautiful Christmas Ornaments! Please add your personalization to the personalization notes section at the checkout. We'll print your info exactly as you provide unless you specifically ask for custom stylized wording/text in a separate message. Proofs are available upon request. Please note - the ornament is not a globe; it is flat. * One-sided clear acrylic ornament * Artwork will not scratch or fade * Artwork is printed on one side * Measures: 3.5"" diameter and 1/8"" thickness * The ornament comes with a beautiful light brown velvet ribbon. These are custom-made ornaments; please allow up to 4-6 days for production. During the Holiday season, please allow 6-8 days for production. Your order will ship with USPS First Class Mail service. We do not accept returns or refunds on customized ornaments. All sales are final. ============== Thank you for visiting Hello Delicious sh

%md 
## Create a Vector Store

In this step, you will compute embeddings for the dataset containing information about the products and store them in a Vector Search index using Databricks Vector Search.

**🚨IMPORTANT: Vector Search endpoints must be created before running the rest of the demo. These are already created for you in Databricks Lab environment.**


In [0]:
# assign vs search endpoint by username
vs_endpoint_prefix = "vs_endpoint_"
vs_endpoint_name = vs_endpoint_prefix+str(get_fixed_integer(DA.unique_name("_")))
print(f"Assigned Vector Search endpoint name: {vs_endpoint_name}.")

Assigned Vector Search endpoint name: vs_endpoint_2.


In [0]:
# Index table name
vs_index_table_fullname = f"{DA.catalog_name}.{DA.schema_name}.product_embeddings"

# Store embeddings in vector store
create_vs_index(vs_endpoint_name, vs_index_table_fullname, vs_source_table_fullname, "product")

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Endpoint named vs_endpoint_2 is ready.
Creating index dbacademy.labuser10152510_1746020930.product_embeddings on endpoint vs_endpoint_2...
Waiting for index to be ready, this can take a few min... {'detailed_state': 'PROVISIONING_INDEX', 'message': 'Delta sync Index creation is pending. Check latest status: https://dbc-7aad3b7d-2c13.cloud.databricks.com/explore/data/dbacademy/labuser10152510_1746020930/product_embeddings', 'indexed_row_count': 0, 'ready': False, 'index_url': 'dbc-7aad3b7d-2c13.cloud.databricks.com/api/2.0/vector-search/indexes/dbacademy.labuser10152510_1746020930.product_embeddings'} - pipeline url:dbc-7aad3b7d-2c13.cloud.databricks.com/api/2.0/vector-search/indexes/dbacademy.labuser10152510_1746020930.product_embeddings


## Task 1: Build the First Chain (Vector Store Search)

In this task, you will create first chain that will search for product details from the Vector Store using a dataset containing product descriptions.

**Instructions:**
   - Configure components for the first chain to perform a search using the Vector Store.
   - Utilize the loaded dataset to generate prompts for Vector Store search queries.
   - Set up retrieval to extract relevant product details based on the generated prompts and search results.


In [0]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import PromptTemplate
from langchain_databricks import ChatDatabricks, DatabricksVectorSearch
from langchain_core.output_parsers import StrOutputParser

# Define the Databricks Chat model: Llama
llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens=1000)

# Define the prompt template for generating search queries
prompt_template_vs = PromptTemplate.from_template(
    """
    You are a hand-crafted product design expert and your task is to creative products that are very good and can be sold online.

    Write a product title and description that is similar to the following product title and item details.

    Maximum 300 words.

    Use the following product title and details as example;

    <context>
    {context}
    </context>

    Question: {input}
    """
)

# Construct the RetrievalQA chain for Vector Store search
def get_retriever(persist_dir=None):
    vsc = VectorSearchClient(disable_notice=True)
    vs_index = vsc.get_index(vs_endpoint_name, vs_index_table_fullname)
    vectorstore = DatabricksVectorSearch(vs_index_table_fullname)
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# Construct the chain for question-answering
question_answer_chain = create_stuff_documents_chain(llm_llama, prompt_template_vs)
chain1 = create_retrieval_chain(get_retriever(), question_answer_chain)

# Invoke the chain with an example query   
response = chain1.invoke({"input": "Query testing product with mobile app control"})
print(response['answer'])

/home/spark-0d77f56b-c76a-4ac4-b345-b7/.ipykernel/14883/command-3951983942472510-3478874156:8: LangChainDeprecationWarning: Use databricks_langchain.ChatDatabricks
  llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens=1000)
/home/spark-0d77f56b-c76a-4ac4-b345-b7/.ipykernel/14883/command-3951983942472510-3478874156:33: LangChainDeprecationWarning: Use databricks_langchain.DatabricksVectorSearch
  vectorstore = DatabricksVectorSearch(vs_index_table_fullname)


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
## Title: Smart Home Automation System with Mobile App Control

## Description: 
Take control of your home with our innovative Smart Home Automation System, designed to make your life easier and more convenient. This system allows you to control and monitor your home's lighting, temperature, security, and entertainment systems remotely using our intuitive mobile app. 

With our system, you can:
- Control your home's lighting, adjusting brightness and color to suit your mood and needs.
- Regulate your home's temperature, ensuring a comfortable environment while saving energy.
- Monitor your home's security, receiving alerts and viewing live footage from your security cameras.
- Stream your favorite music and videos to any room in your house, creating the pe

## Task 2: Build the Second Chain (Optimization)

In this step, you will create a second chain to enhance the product details generated by the first chain. This optimization process aims to make the descriptions more compelling and SEO-friendly. In a real-world scenario, this model could be trained on your internal data or fine-tuned to align with your specific business objectives.

**Instructions:**

- Define a second chain using `llama-3-70b-instruct`.  

- Create a prompt to optimize the generated product description. For example:  
  *"You are a marketing expert. Revise the product title and description to be SEO-friendly and more appealing to Databricks users."*

- Use `product_details` as the parameter to be passed into the prompt.  

- Implement the chain and test it with a sample input.  


In [0]:
# Define the Databricks Chat model: Llama
llm_llama3 = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens=1000)

# Define the prompt template for generating search queries
quality_optimization_prompt = PromptTemplate.from_template(
    """
    You are marketing expert. Change the title and description of the following product to be SEO friendly and make it more attractive to Databricks users.

    Product details: {product_details}

    Answer should include only title and decsription as;
    Title: 
    Descrription:
    """
)

chain2 = quality_optimization_prompt | llm_llama3 | StrOutputParser()
chain2.invoke({"product_details": "Query testing product with mobile app control"})

'Title: Databricks Query Testing & Optimization Tool - Mobile App Control for Data Engineers\n\nDescription: Streamline your Databricks query testing and optimization process with our intuitive mobile app control, designed to help data engineers and analysts work more efficiently. Easily test, validate, and refine your queries on-the-go, ensuring data accuracy and performance. Our tool integrates seamlessly with Databricks, allowing you to identify bottlenecks, reduce costs, and improve overall data quality, all from the convenience of your mobile device.'

## Task 3: Integrate Chains into a Multi-chain System

In this task, you will link the individual chains created in Task 2 and Task 3 together to form a multi-chain system that can handle multi-stage reasoning.

**Instructions:**

- Use Databricks **`Llama Chat model`** for processing text inputs, which is defined above in the first task.

- Create a prompt template to generate an **`HTML page`** for displaying generated product details.

- Construct the **`Multi-Chain System`**  by combining the outputs of the previous chains. **Important**: You will need to rename the out of the first chain and second chain while passing them to the next stage. This sequential chain should be as; **chain3 = chain1 > (`product_details`) > chain2 > `(optimized_product_details)` > prompt3**.  

- Invoke the multi-chain system with the input data to generate the HTML page for the specified product.


In [0]:
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from IPython.display import display, HTML

# Define the prompt template for generating the HTML page
prompt_template_3 = PromptTemplate.from_template(
    """Create an HTML page for the following product details:
    
    Product Description: {optimized_product_details}

    Return a valid HTML code. Don't include html head or body tags.
    """
)

# Construct the multi-chain system
chain3 = (
    chain1 | 
    RunnableMap({"product_details": lambda x: x["answer"]}) |
    chain2|
    RunnableMap({"optimized_product_details": lambda x: x}) |
    prompt_template_3
    | llm_llama 
    | StrOutputParser()
)

# Invoke the chain with question and query
query = {
    "input": "Custom embroidered blanket with mobile app control."      
}
output_html = chain3.invoke(query)

# Display the generated HTML output
display(HTML(output_html))

## Task 4: Save the Chain to Model Registry in UC

In this task, you will save the multi-stage chain system within our Unity Catalog.

**Instructions:**

- Set the model registry to UC and use the model name defined.

- Log and register the final multi-chain system.

- To test the registered model, load the model back from model registry and query it using a sample query. 

After registering the chain, you can view the chain and models in the **Catalog Explorer**.

In [0]:
from mlflow.models import infer_signature
import mlflow

# Set model registry to UC
mlflow.set_registry_uri("databricks-uc")
model_name = f"{DA.catalog_name}.{DA.schema_name}.multi_stage_lab"

with mlflow.start_run(run_name="multi_stage_lab") as run:
    signature = infer_signature(query, output_html)
    model_info = mlflow.langchain.log_model(
        chain3,
        loader_fn=get_retriever, 
        artifact_path="chain",
        registered_model_name=model_name,
        input_example=query,
        signature=signature
    )

model_uri = f"models:/{model_name}/{model_info.registered_model_version}"
model = mlflow.langchain.load_model(model_uri)

output_html = model.invoke(query)
display(HTML(output_html))

/local_disk0/.ephemeral_nfs/envs/pythonEnv-0d77f56b-c76a-4ac4-b345-b705c511952a/lib/python3.10/site-packages/mlflow/langchain/runnables.py:292: UserWarning: Your model contains a class imported from the LangChain partner package `langchain-databricks`. When loading the model back, MLflow will use the community version of the classes instead of the partner packages, which may lead to unexpected behavior. To ensure that the model is loaded correctly, it is recommended to save the model with the 'model-from-code' method instead: https://mlflow.org/docs/latest/models.html#models-from-code
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-0d77f56b-c76a-4ac4-b345-b705c511952a/lib/python3.10/site-packages/mlflow/langchain/runnables.py:292: UserWarning: Your model contains a class imported from the LangChain partner package `langchain-databricks`. When loading the model back, MLflow will use the community version of the classes instead of the partner packages, which may lead to unexp

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Successfully registered model 'dbacademy.labuser10152510_1746020930.multi_stage_lab'.
Created version '1' of model 'dbacademy.labuser10152510_1746020930.multi_stage_lab'.


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().



## Conclusion

In this lab, you've learned how to build a multi-stage AI system using Databricks and LangChain. By integrating multiple chains, you can perform complex reasoning tasks such as searching for product details and optimizing the response based on your business needs. This approach enables the development of sophisticated AI systems capable of handling diverse tasks efficiently.



&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>